In [ ]:
# Alex Yu
# CS498
# Data Analysis

In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 9.4 MB 6.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 4.6 MB 7.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 3.2 MB 6.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 873 kB 6.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/c

In [ ]:
! pip install torch-geometric-temporal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 4.1 MB/s 
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86745 sha256=93c92526a5623cce455b17e74336479471fb819657c97cf5c3f22f5f5ee672d5
  Stored in directory: /root/.cache/pip/wheels/e4/8f/01/24ea76086e0de1805f01b8a62166172c110854128f345d29e9
Successfully built torch-geometric-temporal


In [ ]:
from torch_geometric_temporal.dataset import WindmillOutputSmallDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = WindmillOutputSmallDatasetLoader()
dataset = loader.get_dataset()

**What is the dataset?**

Answer: Hourly energy output of windmills from a European country
    for more than 2 years. Vertices represent 11 windmills and
    weighted edges describe the strength of relationships. The target
    variable allows for regression tasks

**How many data points?**

In [ ]:
print("Number of data points: ", dataset.snapshot_count)
snapshot = dataset[0]

Number of data points:  17464


**What does a single datapoint look like?**

In [ ]:
snapshot

Data(x=[11, 8], edge_index=[2, 121], edge_attr=[121], y=[11])

In [ ]:
print("Features: ", snapshot.num_features, "Edge index: ", len(snapshot.edge_index), "Edge attributes: ", len(snapshot.edge_attr))

Features:  8 Edge index:  2 Edge attributes:  121


In [ ]:
dataset.

AttributeError: ignored

**What are the edge indices?**

Answer: A PyTorch LongTensor of edge indices used for node feature aggregation

**What are the edge attributes?**

Answer: A PyTorch FloatTensor of edge features used for weighting the node feature aggregation, i.e. edge weight

**What kind of task is this - regression or classification?**

Answer: Regression.

In [ ]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvLSTM

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvLSTM(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h


In [ ]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 8)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("MSE: {:.4f}".format(cost))

  4%|▍         | 1/25 [01:02<24:59, 62.47s/it]

MSE: 1.0229


  8%|▊         | 2/25 [02:02<23:20, 60.88s/it]

MSE: 1.0129


 12%|█▏        | 3/25 [03:02<22:17, 60.77s/it]

...

MSE: 1.0021


 96%|█████████▌| 24/25 [24:08<01:00, 60.32s/it]

MSE: 1.0019


100%|██████████| 25/25 [25:08<00:00, 60.34s/it]

MSE: 1.0019


**What is MSE?**

Answer: Mean Squared Error

In [ ]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9830


In [ ]:
class GraphCRN(torch.nn.Module):
    def __init__(self, node_features):
        super(GraphCRN, self).__init__()
        self.recurrent = GConvLSTM(node_features, 32, 7) # 7 is the kernel filter size
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [ ]:
model2 = GraphCRN(node_features = 8)

# use same optimization method but smaller learning rate
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.001)

model2.train()

# keep the number of epochs same
for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer2.step()
    optimizer2.zero_grad()
    print("MSE: {:.4f}".format(cost))

  4%|▍         | 1/25 [01:02<24:57, 62.40s/it]

MSE: 1.0019


  8%|▊         | 2/25 [02:02<23:24, 61.08s/it]

MSE: 1.0018


 12%|█▏        | 3/25 [03:04<22:30, 61.36s/it]

...

MSE: 1.0009


 96%|█████████▌| 24/25 [24:08<00:59, 59.62s/it]

MSE: 1.0008


100%|██████████| 25/25 [25:08<00:00, 60.33s/it]

MSE: 1.0008


In [ ]:
model2.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model2(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9951


In [ ]:
# add a drop-out layer
class DropCRN(torch.nn.Module):
    def __init__(self, node_features):
        super(DropCRN, self).__init__()
        self.recurrent = GConvLSTM(node_features, 32, 4) # 4 is the kernel filter size
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h, c = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = F.dropout(h, training=self.training)
        h = self.linear(h)
        return h

In [ ]:
model3 = GraphCRN(node_features = 8)

optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.01)

model3.train()

# increase the number of epochs to 100 (same as the paper)
for epoch in tqdm(range(100)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model3(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer3.step()
    optimizer3.zero_grad()
    print("MSE: {:.4f}".format(cost))

  1%|          | 1/100 [03:21<5:32:49, 201.71s/it]

MSE: 1.0443


  2%|▏         | 2/100 [06:45<5:30:59, 202.65s/it]

MSE: 1.0191


  3%|▎         | 3/100 [10:02<5:23:41, 200.23s/it]

...

 67%|██████▋   | 67/100 [3:39:13<1:47:00, 194.56s/it]

MSE: 0.9743


 68%|██████▊   | 68/100 [3:42:30<1:44:13, 195.42s/it]

MSE: 0.9736


 69%|██████▉   | 69/100 [3:45:49<1:41:22, 196.21s/it]

In [ ]:
model3.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model3(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

In [ ]:
for epoch in tqdm(range(25)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model3(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer3.step()
    optimizer3.zero_grad()
    print("MSE: {:.4f}".format(cost))